# **IEOR 4571 Fall 2020 Homework2 Report**


- Hu, Bo (uni: bh2569)
- Qin, Rui (uni: rq217)
- Yuan, Shuibenyang (uni: sy2938)

In [1]:
%load_ext autoreload
%autoreload 2 
import sys
sys.path.append('../')

# Objective

# The Sample

In [2]:
# todo how did we sample the data

In [3]:
from src.utils import loading, Spark
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd
import numpy as np

## Load Sample

In [4]:
# create spark session
spark = Spark()

Spark UI address http://127.0.0.1:4040


In [5]:
# load sample data from '/data/raw/sample.csv'
datas = loading(spark, '../data/raw')
sample = datas['sample']

## Sample Objectives

In [ ]:
print(f'''
            number of data points in the sample: {sample.count()},
            number of unique users in the sample: {sample.select('userId').distinct().count()},
            number of unique movies in the sample: {sample.select('movieId').distinct().count()},
            mean of number of movies a user rated:{sample.groupby('userId').agg(F.count('movieId').alias('cnt')).select(F.mean('cnt')).collect()[0][0]},
            mean of number of users a movie be rated: {sample.groupby('movieId').agg(F.count('userId').alias('cnt')).select(F.mean('cnt')).collect()[0][0]},
            average rating: {sample.select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating: {sample.select(F.stddev('rating')).collect()[0][0]},
            average rating by user: {sample.groupby('userId').agg(F.mean('rating').alias('rating')).select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating by user mean: {sample.groupby('userId').agg(F.mean('rating').alias('rating')).select(F.stddev('rating')).collect()[0][0]},
            average rating by movie: {sample.groupby('movieId').agg(F.mean('rating').alias('rating')).select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating by movie mean: {sample.groupby('movieId').agg(F.mean('rating').alias('rating')).select(F.stddev('rating')).collect()[0][0]}
        ''')

# Evaluation

## Cross Validation Setup

In [7]:
from src.evaluation import Evaluator, Cross_validate_als
from src.model_based import Als
import time

In [8]:
splits = loading(spark, '../data/interim')

In [ ]:
print(list(splits.keys()))

In [9]:
# build evaluation pipeline
def evaluate(train, test, evaluators, model):
    print('training')
    start = time.time()
    model.fit(train)
    end = time.time()
    print(f'training time: {round(end - start, 2)} seconds')
    print('inferencing train set')
    start = time.time()
    train_pred = model.predict(train)
    end = time.time()
    print(f'inference time: {round(end - start, 2)} seconds')
    print('inferencing test set')
    start = time.time()
    test_pred = model.predict(test)
    end = time.time()
    print(f'inference time: {round(end - start, 2)} seconds')
    res = pd.DataFrame(np.zeros((len(evaluators),2)), columns = ['train', 'test'], index = evaluators.keys())
    for eva in evaluators.keys():
        res.loc[eva, 'train'] = evaluators[eva].evaluate(train_pred)
        res.loc[eva, 'test'] = evaluators[eva].evaluate(test_pred)
    return res

## Metrics

In [10]:
evaluators = {'rmse': Evaluator(metrics = 'rmse'), 
              'accuracy': Evaluator(metrics = 'accuracy'), 
              'coverage_2': Evaluator(metrics = 'converage_k', 
                                       ratingCol='rating', 
                                       predCol='prediction', 
                                       idCol='userId', 
                                       k=2)}

## Memory Based Collaborative Filtering

In [12]:
from src.memory_based import Memory_based_CF

### User Based Collaborative Filtering

In [13]:
cf = Memory_based_CF(spark, 'user')

In [14]:
%%time
result = []
for i in ['0.75_0.25', '0.5_0.5', '0.25_0.75']:
    train, test = splits['train_' + i], splits['test_' + i]
    res = evaluate(train, test, evaluators, cf)
    result.append(res)
ub_CF_res = pd.DataFrame(index=['rmse', 'accuracy', 'coverage_5'], columns = ['train', 'test', 'split'])
for i, j in zip(result, ['0.75_0.25', '0.5_0.5', '0.25_0.75']):
    i['split'] = j
    ub_CF_res = ub_CF_res.append(i)
ub_CF_res = ub_CF_res.dropna()

training
training time: 37.99 seconds
inferencing train set
inference time: 11.17 seconds
inferencing test set
inference time: 3.86 seconds
training
training time: 31.22 seconds
inferencing train set
inference time: 7.7 seconds
inferencing test set
inference time: 7.34 seconds
training
training time: 26.34 seconds
inferencing train set
inference time: 3.85 seconds
inferencing test set
inference time: 9.92 seconds
CPU times: user 3min 46s, sys: 25.7 s, total: 4min 12s
Wall time: 2min 25s


In [20]:
ub_CF_res

,train,test,split
rmse,0.822981,0.882472,0.75_0.25
accuracy,0.737826,0.705916,0.75_0.25
coverage_2,0.931907,0.707219,0.75_0.25
rmse,0.812562,0.903506,0.5_0.5
accuracy,0.744819,0.695154,0.5_0.5
coverage_2,0.880891,0.861922,0.5_0.5
rmse,0.789096,0.961355,0.25_0.75
accuracy,0.761302,0.674534,0.25_0.75
coverage_2,0.734384,0.920091,0.25_0.75


In [16]:
ub_CF_res.to_csv('../data/processed/ub_cf_res.csv', header = True, index = True)

### Item Based Collaborative Filtering

In [17]:
cf = Memory_based_CF(spark, 'item')

In [18]:
%%time
result = []
for i in ['0.75_0.25', '0.5_0.5', '0.25_0.75']:
    train, test = splits['train_' + i], splits['test_' + i]
    res = evaluate(train, test, evaluators, cf)
    result.append(res)
ib_CF_res = pd.DataFrame(index=['rmse', 'accuracy', 'coverage_5'], columns = ['train', 'test', 'split'])
for i, j in zip(result, ['0.75_0.25', '0.5_0.5', '0.25_0.75']):
    i['split'] = j
    ib_CF_res = ib_CF_res.append(i)
ib_CF_res = ib_CF_res.dropna()

training
training time: 6.08 seconds
inferencing train set
inference time: 10.81 seconds
inferencing test set
inference time: 4.01 seconds
training
training time: 5.71 seconds
inferencing train set
inference time: 7.41 seconds
inferencing test set
inference time: 6.94 seconds
training
training time: 4.19 seconds
inferencing train set
inference time: 3.73 seconds
inferencing test set
inference time: 10.19 seconds
CPU times: user 55.1 s, sys: 1.77 s, total: 56.9 s
Wall time: 1min 3s


In [19]:
ib_CF_res

,train,test,split
rmse,0.852612,0.854921,0.75_0.25
accuracy,0.723621,0.722746,0.75_0.25
coverage_2,0.910622,0.691240,0.75_0.25
rmse,0.876075,0.876476,0.5_0.5
accuracy,0.717357,0.714118,0.5_0.5
coverage_2,0.865931,0.840369,0.5_0.5
rmse,0.919891,0.935865,0.25_0.75
accuracy,0.707726,0.697115,0.25_0.75
coverage_2,0.736767,0.898118,0.25_0.75


In [21]:
ib_CF_res.to_csv('../data/processed/ib_cf_res.csv', header = True, index = True)

## Model Based Collaborative Filtering

### ALS Matrix Factorization Approximation

In [90]:
from src.evaluation import Cross_validate_als

#### Parameter Tuning for ALS

In [45]:
parameters = {
    "regParam": [0.01, 0.05, 0.1, 0.15],
    "rank": [10, 50, 100, 150, 200]
}

cf = cf = Als(userCol='userId',
                itemCol='movieId',
                ratingCol='rating',
                regParam=.15,
                seed=0,
                rank=10
                )

In [46]:
result = []
for i in ['0.75_0.25', '0.5_0.5', '0.25_0.75']:
    train, test = splits['train_' + i], splits['test_' + i]
    res = Cross_validate_als(training = train,
                            test= test,
                            valid_ratio = .1,
                            regParam = parameters['regParam'],
                            rank = parameters['rank'],
                            seed = 0,
                            evaluators = list(evaluators.values()))
    result.append(res)

100%|██████████| 20/20 [06:39<00:00, 19.95s/it]


In [59]:
als_parameter_tuning = pd.DataFrame(columns = ['rmse', 'accuracy', 'converage_k'])
for i, j in zip(result, ['0.75_0.25', '0.5_0.5', '0.25_0.75']):
    i['split'] = j
    als_parameter_tuning = als_parameter_tuning.append(i)
als_parameter_tuning = als_parameter_tuning.dropna()
als_parameter_tuning.index.name = '(regParam, rank)'
als_parameter_tuning.to_csv('../data/processed/als_parameter_tuning.csv', header=True, index=True)

In [81]:
print(f'the best parameter for each split and metric is')
print(f'best param for accuracy')
display(als_cv_res.groupby('split').rmse.apply(lambda x: x.idxmin()))
print(f'best param for rmse')
display(als_cv_res.groupby('split').accuracy.apply(lambda x: x.idxmax()))
print(f'best param for coverage_k')
display(als_cv_res.groupby('split').accuracy.apply(lambda x: x.idxmax()))

the best parameter for each split and metric is
best param for accuracy


split
0.25_0.75    (0.15, 10)
0.5_0.5      (0.15, 10)
0.75_0.25    (0.15, 10)
Name: rmse, dtype: object

best param for rmse


split
0.25_0.75    (0.15, 10)
0.5_0.5      (0.15, 10)
0.75_0.25    (0.15, 10)
Name: accuracy, dtype: object

best param for coverage_k


split
0.25_0.75    (0.15, 10)
0.5_0.5      (0.15, 10)
0.75_0.25    (0.15, 10)
Name: accuracy, dtype: object

We will choose the best parameter regParam = .15, rank = 10

#### ALS evaluation

In [82]:
from src.model_based import Als

In [83]:
cf = Als(userCol='userId',
                itemCol='movieId',
                ratingCol='rating',
                regParam=.15,
                seed=0,
                rank=10
                )

In [84]:
%%time
result = []
for i in ['0.75_0.25', '0.5_0.5', '0.25_0.75']:
    train, test = splits['train_' + i], splits['test_' + i]
    res = evaluate(train, test, evaluators, cf)
    result.append(res)
als_cf_res = pd.DataFrame(index=['rmse', 'accuracy', 'coverage_5'], columns = ['train', 'test', 'split'])
for i, j in zip(result, ['0.75_0.25', '0.5_0.5', '0.25_0.75']):
    i['split'] = j
    als_cf_res = als_cf_res.append(i)
als_cf_res = als_cf_res.dropna()

training
training time: 1.87 seconds
inferencing train set
inference time: 0.01 seconds
inferencing test set
inference time: 0.02 seconds
training
training time: 1.78 seconds
inferencing train set
inference time: 0.01 seconds
inferencing test set
inference time: 0.02 seconds
training
training time: 1.4 seconds
inferencing train set
inference time: 0.02 seconds
inferencing test set
inference time: 0.01 seconds
CPU times: user 92.8 ms, sys: 22 ms, total: 115 ms
Wall time: 21.1 s


In [86]:
als_cf_res.to_csv('../data/processed/als_result.csv', header=True, index=True)

In [87]:
als_cf_res

,train,test,split
rmse,0.708663,0.868133,0.75_0.25
accuracy,0.746797,0.670438,0.75_0.25
coverage_2,0.906212,0.658363,0.75_0.25
rmse,0.658669,0.910945,0.5_0.5
accuracy,0.766557,0.649448,0.5_0.5
coverage_2,0.844469,0.800571,0.5_0.5
rmse,0.550226,1.010943,0.25_0.75
accuracy,0.805711,0.609509,0.25_0.75
coverage_2,0.690452,0.850068,0.25_0.75
